In [2]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
import requests
import steamspypi

# customisations - ensure tables show all columns
pd.set_option("max_columns", 100)

In [3]:
def get_request(url, parameters=None):
    """Return json-formatted response of a get request using optional parameters.
    
    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request
    
    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except SSLError as s:
        print('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)
        
        # recusively try again
        return get_request(url, parameters)
    
    if response:
        return response.json()
    else:
        # response is none usually means too many requests. Wait and try again 
        print('No response, waiting 10 seconds...')
        time.sleep(10)
        print('Retrying.')
        return get_request(url, parameters)

In [11]:
url = "https://steamspy.com/api.php"
parameters = {"request": "all"}

# request 'all' from steam spy and parse into dataframe
#json_data = steamspypi.download_all_pages(num_pages=44)
#steam_spy_all = pd.DataFrame.from_dict(json_data, orient='index')

# generate sorted app_list from steamspy data
#app_list = steam_spy_all[['appid', 'name']].sort_values('appid').reset_index(drop=True)

# export disabled to keep consistency across download sessions
#app_list.to_csv('games.csv', index=False)

# instead read from stored csv
app_list = pd.read_csv('games.csv')

# display first few rows
app_list.head()

,appid,name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force


In [33]:
def get_app_data(start, stop, parser, pause):
    """Return list of app data generated from parser.
    
    parser : function to handle request
    """
    app_data = []
    
    # iterate through each row of app_list, confined by start and stop
    for index, row in app_list[start:stop].iterrows():
        print('Current index: {}'.format(index), end='\r')
        
        appid = row['appid']
        name = row['name']

        # retrive app data for a row, handled by supplied parser, and append to list
        data = parser(appid, name)
        app_data.extend(data)

        time.sleep(pause) # prevent overloading api with requests
    
    return app_data


def process_batches(parser, app_list, download_path, data_filename, index_filename,
                    columns, begin=0, end=-1, batchsize=100, pause=1):
    """Process app data in batches, writing directly to file.
    
    parser : custom function to format request
    app_list : dataframe of appid and name
    download_path : path to store data
    data_filename : filename to save app data
    index_filename : filename to store highest index written
    columns : column names for file
    
    Keyword arguments:
    
    begin : starting index (get from index_filename, default 0)
    end : index to finish (defaults to end of app_list)
    batchsize : number of apps to write in each batch (default 100)
    pause : time to wait after each api request (defualt 1)
    
    returns: none
    """
    print('Starting at index {}:\n'.format(begin))
    
    # by default, process all apps in app_list
    if end == -1:
        end = len(app_list) + 1
    
    # generate array of batch begin and end points
    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)
    
    apps_written = 0
    batch_times = []
    
    for i in range(len(batches) - 1):
        start_time = time.time()
        
        start = batches[i]
        stop = batches[i+1]
        
        app_data = get_app_data(start, stop, parser, pause)
        
        rel_path = os.path.join(download_path, data_filename)
        
        # writing app data to file
        with open(rel_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')
            
            for j in range(3,0,-1):
                print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                time.sleep(0.5)
            
            writer.writerows(app_data)
            print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')
            
        apps_written += len(app_data)
        
        idx_path = os.path.join(download_path, index_filename)
        
        # writing last index to file
        with open(idx_path, 'w') as f:
            index = stop
            print(index, file=f)
            
        # logging time taken
        end_time = time.time()
        time_taken = end_time - start_time
        
        batch_times.append(time_taken)
        mean_time = statistics.mean(batch_times)
        
        est_remaining = (len(batches) - i - 2) * mean_time
        
        remaining_td = dt.timedelta(seconds=round(est_remaining))
        time_td = dt.timedelta(seconds=round(time_taken))
        mean_td = dt.timedelta(seconds=round(mean_time))
        
        print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
            
    print('\nProcessing batches complete. {} apps written'.format(apps_written))

In [6]:
def reset_index(download_path, index_filename):
    """Reset index in file to 0."""
    rel_path = os.path.join(download_path, index_filename)
    
    with open(rel_path, 'w') as f:
        print(0, file=f)
        

def get_index(download_path, index_filename):
    """Retrieve index from file, returning 0 if file not found."""
    try:
        rel_path = os.path.join(download_path, index_filename)

        with open(rel_path, 'r') as f:
            index = int(f.readline())
    
    except FileNotFoundError:
        index = 0
        
    return index


def prepare_data_file(download_path, filename, index, columns):
    """Create file and write headers if index is 0."""
    if index == 0:
        rel_path = os.path.join(download_path, filename)

        with open(rel_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()

In [18]:
def parse_steam_request(appid, name):
    """Unique parser to handle data from Steam Store API.
    
    Returns : json formatted data (dict-like)
    """
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": appid}
    
    json_data = get_request(url, parameters=parameters)
    json_app_data = json_data[str(appid)]
    
    if json_app_data['success']:
        data = json_app_data['data']
    else:
        data = {'name': name, 'steam_appid': appid}
        
    return data


# Set file parameters
download_path = 'gameDownload'
steam_app_data = 'steam_app_data.csv'
steam_index = 'steam_app_index.txt'

steam_columns = [
    'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
    'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
    'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
    'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
    'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
    'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
    'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
    'background', 'content_descriptors'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
#reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if index is 0
prepare_data_file(download_path, steam_app_data, index, steam_columns)

# Set end and chunksize for demonstration - remove to run through entire app list
process_batches(
    parser=parse_steam_request,
    app_list=app_list,
    download_path=download_path,
    data_filename=steam_app_data,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index,
    end=16634,
    batchsize=5
)

Starting at index 12170:

Exported lines 12170-12174 to steam_app_data.csv. Batch 0 time: 0:00:09 (avg: 0:00:09, remaining: 2:07:28)
Exported lines 12175-12179 to steam_app_data.csv. Batch 1 time: 0:00:08 (avg: 0:00:08, remaining: 2:02:51)
Exported lines 12180-12184 to steam_app_data.csv. Batch 2 time: 0:00:08 (avg: 0:00:08, remaining: 2:01:55)
Exported lines 12185-12189 to steam_app_data.csv. Batch 3 time: 0:00:08 (avg: 0:00:08, remaining: 2:00:47)
Exported lines 12190-12194 to steam_app_data.csv. Batch 4 time: 0:00:08 (avg: 0:00:08, remaining: 2:00:31)
Exported lines 12195-12199 to steam_app_data.csv. Batch 5 time: 0:00:08 (avg: 0:00:08, remaining: 2:00:08)
Exported lines 12200-12204 to steam_app_data.csv. Batch 6 time: 0:00:08 (avg: 0:00:08, remaining: 1:59:59)
Exported lines 12205-12209 to steam_app_data.csv. Batch 7 time: 0:00:08 (avg: 0:00:08, remaining: 1:59:48)
Exported lines 12210-12214 to steam_app_data.csv. Batch 8 time: 0:00:08 (avg: 0:00:08, remaining: 1:59:31)
Exported li

In [ ]:
# inspect downloaded data
pd.read_csv('../data/download/steam_app_data.csv').head()

In [15]:
def parse_steamspy_request(appid, name):
    """Parser to handle SteamSpy API data."""
    url = "https://steamspy.com/api.php"
    parameters = {"request": "appdetails", "appid": appid}
    
    json_data = get_request(url, parameters)
    return json_data


# set files and columns
download_path = 'gameDownload'
steamspy_data = 'steamspy_data.csv'
steamspy_index = 'steamspy_index.txt'

steamspy_columns = [
    'appid', 'name', 'developer', 'publisher', 'score_rank', 'positive',
    'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
    'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
    'languages', 'genre', 'ccu', 'tags'
]

#reset_index(download_path, steamspy_index)
index = get_index(download_path, steamspy_index)

# Wipe data file if index is 0
prepare_data_file(download_path, steamspy_data, index, steamspy_columns)

process_batches(
    parser=parse_steamspy_request,
    app_list=app_list,
    download_path=download_path, 
    data_filename=steamspy_data,
    index_filename=steamspy_index,
    columns=steamspy_columns,
    begin=index,
    end=16634,
    batchsize=5,
    pause=0.3
)

Starting at index 10:

Exported lines 10-14 to steamspy_data.csv. Batch 0 time: 0:00:05 (avg: 0:00:05, remaining: 4:47:29)
Exported lines 15-19 to steamspy_data.csv. Batch 1 time: 0:00:05 (avg: 0:00:05, remaining: 4:43:51)
Exported lines 20-24 to steamspy_data.csv. Batch 2 time: 0:00:05 (avg: 0:00:05, remaining: 4:41:38)
Exported lines 25-29 to steamspy_data.csv. Batch 3 time: 0:00:05 (avg: 0:00:05, remaining: 4:39:25)
Exported lines 30-34 to steamspy_data.csv. Batch 4 time: 0:00:05 (avg: 0:00:05, remaining: 4:37:59)
Exported lines 35-39 to steamspy_data.csv. Batch 5 time: 0:00:05 (avg: 0:00:05, remaining: 4:34:44)
Exported lines 40-44 to steamspy_data.csv. Batch 6 time: 0:00:05 (avg: 0:00:05, remaining: 4:35:11)
Exported lines 45-49 to steamspy_data.csv. Batch 7 time: 0:00:05 (avg: 0:00:05, remaining: 4:32:27)
Exported lines 50-54 to steamspy_data.csv. Batch 8 time: 0:00:05 (avg: 0:00:05, remaining: 4:33:33)
Exported lines 55-59 to steamspy_data.csv. Batch 9 time: 0:00:05 (avg: 0:00:0

In [19]:
# inspect downloaded steamspy data
pd.read_csv('steamspy_data.csv').head()

EmptyDataError: No columns to parse from file

In [39]:
def get_n_reviews(appid, name):
    
    url = "https://store.steampowered.com/appreviews/"
    url = url + str(appid)
    
    reviews = []
    cursor = '*'
    params = {
            'json' : 1,
            'filter' : 'all',
            'language' : 'english',
            'day_range' : 9223372036854775807,
            'review_type' : 'all',
            'purchase_type' : 'all'
            }

   
    params['cursor'] = cursor.encode()
    params['num_per_page'] = 100

    response = get_request(url, params)
    cursor = response['cursor']
        
    if response['success']:
        #response['reviews']['steam_appid'] = appid
        data = response['reviews']
        for review in response['reviews']:
            review['steam_appid'] =appid
        #print(data)
    else:
        data = {'name': name, 'steam_appid': appid}
        
    return data
        

# Set file parameters
download_path = 'gameDownload'
steam_app_data = 'steam_app_reviews.csv'
steam_index = 'steam_app_reviews_index.txt'

steam_columns = [
    'steam_appid', 'review', 'timestamp_created', 'timestamp_updated',
    'voted_up', 'votes_up', 'votes_funny', 'weighted_vote_score', 'comment_count',
    'steam_purchase', 'received_for_free', 'written_during_early_access', 'developer_response', 'timestamp_dev_responded'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if index is 0
prepare_data_file(download_path, steam_app_data, index, steam_columns)

# Set end and chunksize for demonstration - remove to run through entire app list
process_batches(
    parser=get_n_reviews,
    app_list=app_list,
    download_path=download_path,
    data_filename=steam_app_data,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index,
    end=16634,
    batchsize=5
)

Starting at index 0:

Exported lines 0-4 to steam_app_reviews.csv. Batch 0 time: 0:00:09 (avg: 0:00:09, remaining: 8:14:24)
Exported lines 5-9 to steam_app_reviews.csv. Batch 1 time: 0:00:10 (avg: 0:00:09, remaining: 8:31:21)
Exported lines 10-14 to steam_app_reviews.csv. Batch 2 time: 0:00:11 (avg: 0:00:10, remaining: 8:58:33)
Exported lines 15-19 to steam_app_reviews.csv. Batch 3 time: 0:00:09 (avg: 0:00:10, remaining: 8:54:31)
Exported lines 20-24 to steam_app_reviews.csv. Batch 4 time: 0:00:11 (avg: 0:00:10, remaining: 9:03:48)
Exported lines 25-29 to steam_app_reviews.csv. Batch 5 time: 0:00:11 (avg: 0:00:10, remaining: 9:12:35)
Exported lines 30-34 to steam_app_reviews.csv. Batch 6 time: 0:00:10 (avg: 0:00:10, remaining: 9:11:51)
Exported lines 35-39 to steam_app_reviews.csv. Batch 7 time: 0:00:10 (avg: 0:00:10, remaining: 9:13:38)
Exported lines 40-44 to steam_app_reviews.csv. Batch 8 time: 0:00:11 (avg: 0:00:10, remaining: 9:16:37)
Exported lines 45-49 to steam_app_reviews.csv.

KeyboardInterrupt: 